In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Merge all Top 100 Charts into one big panel

1. I create a new dataframe called 'chartsall' containing the following features:
    * artist
    * title
    * year
    * month
    * week
* in_charts - as support feature (gets deleted in the final outcome dataframe)
2. I add the respective features from the first Top 100 Charts list into the new dataframe. If a song has been in the charts for a period longer than the number of weeks in the year 2018, I mark this song as from the year 2017. I also include a new column with the name y2018_1 with the values of the current_rank feature of the first Charts list.
3. I repeat this procedure for all Charts lists of the years 2018-2020. For the year 2018 I filter all songs which have been in the charts for a period longer than the number of weeks at point of release of the respective Charts list in the year 2018, I mark these songs as from the year 2017.
4. I analyze the dataframe and fill all missing values in the columns with the information of the weekly position with "0".
5. Finally I changed all float features into integer features and saved the dataframe.

#### 1.-4. Create dataframe with aggregated data from all Top 100 Charts lists

In [4]:
songinfo = ['artist', 'title', 'year', 'month', 'week', 'in_charts']

col_names = songinfo

chartsall = pd.DataFrame(columns=col_names)

charts_2018_1 = pd.read_pickle("../pickle/chartlists/charts_2018_1.pkl")
charts_2018_1

chartsall['artist'] = charts_2018_1['artist']
chartsall['title'] = charts_2018_1['title']
chartsall['month'] = charts_2018_1['month']
chartsall['y2018_1'] = charts_2018_1['current_rank']
chartsall['week'] = 1
chartsall['in_charts'] = charts_2018_1['in_charts']

year_adjust = []

for x,z in zip(chartsall['week'], chartsall['in_charts']):
    if int(z) > int(x):
        y = 2017
        year_adjust.append(y)
    else:
        y = 2018
        year_adjust.append(y)
        
chartsall['year'] = year_adjust

chartsall = chartsall.drop(['in_charts'], axis=1)
        
#Start appending with all chart lists of 2018

for cw in range(2,53):

    address = "../pickle/chartlists/charts_" + "2018" + "_" + str(cw) + ".pkl"
    weekrank = "y" + "2018" + "_" + str(cw)

    weekcharts = pd.read_pickle(address)
    
    weekcharts['week'] = int(cw)
    
    year_adjust = []
    for x,z in zip(weekcharts['week'], weekcharts['in_charts']):
        if int(z) > int(x):
            y = 2017
            year_adjust.append(y)
        else:
            y = 2018
            year_adjust.append(y)
            
    weekcharts['year'] = year_adjust
  
    chartsall = chartsall.append(weekcharts[['artist', 'title', 'year', 'month']], ignore_index=True, sort=False)

    chartsall = chartsall.drop_duplicates(subset=['artist', 'title'], keep='first', ignore_index=True)

    chartsall['week'] = chartsall['week'].fillna(cw)

    chartsall = pd.merge(chartsall, weekcharts[['artist', 'title', 'current_rank']], how='left', on=['artist', 'title'],
                         sort=False)

    chartsall = chartsall.rename(columns={"current_rank": weekrank})

#Now appending all chart lists of 2019

for cw in range(1,53):

    address = "../pickle/chartlists/charts_" + "2019" + "_" + str(cw) + ".pkl"
    weekrank = "y" + "2019" + "_" + str(cw)

    weekcharts = pd.read_pickle(address)

    chartsall = chartsall.append(weekcharts[['artist', 'title', 'year', 'month']], ignore_index=True, sort=False)

    chartsall = chartsall.drop_duplicates(subset=['artist', 'title'], keep='first', ignore_index=True)

    chartsall['week'] = chartsall['week'].fillna(cw)

    chartsall = pd.merge(chartsall, weekcharts[['artist', 'title', 'current_rank']], how='left', on=['artist', 'title'],
                         sort=False)

    chartsall = chartsall.rename(columns={"current_rank": weekrank})
    
#Finally appending all chart lists of 2020

for cw in range(1,53):

    address = "../pickle/chartlists/charts_" + "2020" + "_" + str(cw) + ".pkl"
    weekrank = "y" + "2020" + "_" + str(cw)

    weekcharts = pd.read_pickle(address)

    chartsall = chartsall.append(weekcharts[['artist', 'title', 'year', 'month']], ignore_index=True, sort=False)

    chartsall = chartsall.drop_duplicates(subset=['artist', 'title'], keep='first', ignore_index=True)

    chartsall['week'] = chartsall['week'].fillna(cw)

    chartsall = pd.merge(chartsall, weekcharts[['artist', 'title', 'current_rank']], how='left', on=['artist', 'title'],
                         sort=False)

    chartsall = chartsall.rename(columns={"current_rank": weekrank})

#### 5.-6. Analyzing the properties of the new dataframe and filling missing values

In [5]:
chartsall['year'].value_counts()

2020    637
2019    632
2018    628
2017    109
Name: year, dtype: int64

In [6]:
chartsall = chartsall[chartsall['year'] > 2017]

In [7]:
chartsall['year'].value_counts()

2020    637
2019    632
2018    628
Name: year, dtype: int64

In [8]:
chartsall.shape

(1897, 161)

In [9]:
chartsall.isna().sum()

artist         0
title          0
year           0
month          0
week           0
y2018_1     1892
y2018_2     1882
y2018_3     1878
y2018_4     1870
y2018_5     1864
y2018_6     1862
y2018_7     1856
y2018_8     1857
y2018_9     1850
y2018_10    1842
y2018_11    1844
y2018_12    1842
y2018_13    1841
y2018_14    1833
y2018_15    1832
y2018_16    1824
y2018_17    1829
y2018_18    1824
y2018_19    1822
y2018_20    1816
y2018_21    1815
y2018_22    1808
y2018_23    1808
y2018_24    1808
y2018_25    1805
y2018_26    1803
y2018_27    1803
y2018_28    1806
y2018_29    1805
y2018_30    1804
y2018_31    1803
y2018_32    1802
y2018_33    1802
y2018_34    1802
y2018_35    1801
y2018_36    1799
y2018_37    1800
y2018_38    1800
y2018_39    1800
y2018_40    1799
y2018_41    1799
y2018_42    1799
y2018_43    1799
y2018_44    1799
y2018_45    1799
y2018_46    1799
y2018_47    1800
y2018_48    1802
y2018_49    1802
y2018_50    1803
y2018_51    1802
y2018_52    1802
y2019_1     1805
y2019_2     18

In [10]:
chartsall = chartsall.fillna(0)

In [11]:
chartsall.isna().sum()

artist      0
title       0
year        0
month       0
week        0
y2018_1     0
y2018_2     0
y2018_3     0
y2018_4     0
y2018_5     0
y2018_6     0
y2018_7     0
y2018_8     0
y2018_9     0
y2018_10    0
y2018_11    0
y2018_12    0
y2018_13    0
y2018_14    0
y2018_15    0
y2018_16    0
y2018_17    0
y2018_18    0
y2018_19    0
y2018_20    0
y2018_21    0
y2018_22    0
y2018_23    0
y2018_24    0
y2018_25    0
y2018_26    0
y2018_27    0
y2018_28    0
y2018_29    0
y2018_30    0
y2018_31    0
y2018_32    0
y2018_33    0
y2018_34    0
y2018_35    0
y2018_36    0
y2018_37    0
y2018_38    0
y2018_39    0
y2018_40    0
y2018_41    0
y2018_42    0
y2018_43    0
y2018_44    0
y2018_45    0
y2018_46    0
y2018_47    0
y2018_48    0
y2018_49    0
y2018_50    0
y2018_51    0
y2018_52    0
y2019_1     0
y2019_2     0
y2019_3     0
y2019_4     0
y2019_5     0
y2019_6     0
y2019_7     0
y2019_8     0
y2019_9     0
y2019_10    0
y2019_11    0
y2019_12    0
y2019_13    0
y2019_14    0
y2019_

In [12]:
for column in chartsall.columns.tolist()[3:len(chartsall.columns.tolist())+1:1]:
    chartsall[column] = chartsall[column].astype(int)

In [13]:
chartsall.dtypes

artist      object
title       object
year         int64
month        int32
week         int32
y2018_1      int32
y2018_2      int32
y2018_3      int32
y2018_4      int32
y2018_5      int32
y2018_6      int32
y2018_7      int32
y2018_8      int32
y2018_9      int32
y2018_10     int32
y2018_11     int32
y2018_12     int32
y2018_13     int32
y2018_14     int32
y2018_15     int32
y2018_16     int32
y2018_17     int32
y2018_18     int32
y2018_19     int32
y2018_20     int32
y2018_21     int32
y2018_22     int32
y2018_23     int32
y2018_24     int32
y2018_25     int32
y2018_26     int32
y2018_27     int32
y2018_28     int32
y2018_29     int32
y2018_30     int32
y2018_31     int32
y2018_32     int32
y2018_33     int32
y2018_34     int32
y2018_35     int32
y2018_36     int32
y2018_37     int32
y2018_38     int32
y2018_39     int32
y2018_40     int32
y2018_41     int32
y2018_42     int32
y2018_43     int32
y2018_44     int32
y2018_45     int32
y2018_46     int32
y2018_47     int32
y2018_48    

In [15]:
chartsall.to_pickle("../../files/pickle/chartsall.pkl")
chartsall.to_csv("../../files/csv/chartsall.csv")